# Week 17, Lecture 01 CodeAlong
- Coefficients & Feature Importance

## Lesson Objectives

- By the end of this lesson, students will be able to:
    - Extract feature names from sklearn v1.1 objects
    - Extract and visualize coefficients
    - Save models to a joblib file


### Life Expectancy

> Task Inspired by: https://medium.com/@shanzehhaji/using-a-linear-regression-model-to-predict-life-expectancy-de3aef66ac21

- Kaggle Dataset on Life Expectancy:
    - https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who

In [1]:
## Our standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as miss

## Preprocessing tools
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

## Models & evaluation metrics
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib

## setting random state for reproducibility
SEED = 321
np.random.seed(SEED)
## Matplotlib style
fav_style = ('ggplot','tableau-colorblind10')
fav_context  ={'context':'notebook', 'font_scale':1.1}
plt.style.use(fav_style)
sns.set_context(**fav_context)
plt.rcParams['savefig.transparent'] = False
plt.rcParams['savefig.bbox'] = 'tight'

In [2]:
## Importing Custom Functions
import sys,os
sys.path.append(os.path.abspath("../"))
%load_ext autoreload
%autoreload 2
from CODE import data_enrichment as de

In [7]:
pd.read_csv('Data/life.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Data/life.csv'

In [6]:
import pandas as pd
df= pd.read_csv('Data/life.csv')
df.info()
df.head(3)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/life.csv'

In [ ]:
# clean extra spaces
df.columns = df.columns.str.strip()
df

In [ ]:
pd.set_option('display.max_columns',0)
df

## EDA

In [ ]:
df.isna().sum()

In [ ]:
miss.matrix(df)

> Can't have null values for the target!

In [ ]:
# drop null values ONLY FROM TARGET
df = df.dropna(subset=['Life expectancy'])

In [ ]:
target = 'Life expectancy'

grid_spec = {'height_ratios':[0.8,0.2]}
fig, axes = plt.subplots(nrows=2, figsize=(6,5), gridspec_kw=grid_spec)

sns.histplot(data=df, x=target,ax=axes[0])
sns.boxplot(data=df, x=target, ax=axes[1]);

## Preprocessing (with Sklearn v1.1+)

In [ ]:
# Run the following command on your local computer to check the version of sklearn
import sklearn
!python --version
print(f"sklearn version: {sklearn.__version__}")

In [ ]:
# ### Train Test Split
## Make x and y variables
target = "Life expectancy"
drop_feats = []

y = df[target].copy()
X = df.drop(columns=[target, *drop_feats]).copy()

## train-test-split with random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=SEED)
X_train.head(3)

In [ ]:
## Make numeric preprocessing pipeline
num_sel = make_column_selector(dtype_include='number')
num_pipe = make_pipeline(SimpleImputer(strategy='mean'))
num_pipe

In [ ]:
## Make categorical preprocessing pipeline
cat_sel = make_column_selector(dtype_include='object')
cat_pipe = make_pipeline(SimpleImputer(strategy='constant',
                                       fill_value='MISSING'),
                         OneHotEncoder(handle_unknown='ignore', sparse=False))

In [ ]:
## make the preprocessing column transformer
preprocessor = make_column_transformer((num_pipe, num_sel),
                                       (cat_pipe,cat_sel),)
preprocessor

### Make X_train_df and X_test_df, dataframe verisons of processed X_train/X_test.

In [ ]:
preprocessor.fit(X_train)
X_train_df = pd.DataFrame(preprocessor.transform(X_train),
                          columns=preprocessor.get_feature_names_out(),
                         index=X_train.index)
X_test_df = pd.DataFrame(preprocessor.transform(X_test),
                          columns=preprocessor.get_feature_names_out(),
                         index=X_test.index)
display(X_train_df)

> #### Q: What's up with the feature names? What are we seeing?

### Get Features Names + Verbose Feature Names Out

In [ ]:
## make the preprocessing column transformer WITH CORRECT ARGS!
preprocessor = make_column_transformer((num_pipe, num_sel),
                                       (cat_pipe,cat_sel),
                                      verbose_feature_names_out=False)
preprocessor.fit(X_train)
X_train_df = pd.DataFrame(preprocessor.transform(X_train),
                          columns=preprocessor.get_feature_names_out(),
                         index=X_train.index)
X_test_df = pd.DataFrame(preprocessor.transform(X_test),
                          columns=preprocessor.get_feature_names_out(),
                         index=X_test.index)
display(X_train_df)


# Modeling - Linear Regression

In [ ]:
def evaluate_regression(model, X_train,y_train, X_test, y_test,for_slides=True): 
    """Evaluates a scikit learn regression model using r-squared and RMSE
    FOR SLIDES VERS DOES MULTIPLE PRINT STATEMENTS FOR VERTICAL DISPLAY OF INFO"""
    
    ## Training Data
    y_pred_train = model.predict(X_train)
    r2_train = metrics.r2_score(y_train, y_pred_train)
    rmse_train = metrics.mean_squared_error(y_train, y_pred_train, 
                                            squared=False)
    mae_train = metrics.mean_absolute_error(y_train, y_pred_train)
    

    ## Test Data
    y_pred_test = model.predict(X_test)
    r2_test = metrics.r2_score(y_test, y_pred_test)
    rmse_test = metrics.mean_squared_error(y_test, y_pred_test, 
                                            squared=False)
    mae_test = metrics.mean_absolute_error(y_test, y_pred_test)
    
    if for_slides:
        df_version =[['Split','R^2','MAE','RMSE']]
        df_version.append(['Train',r2_train, mae_train, rmse_train])
        df_version.append(['Test',r2_test, mae_test, rmse_test])
        df_results = pd.DataFrame(df_version[1:], columns=df_version[0])
        df_results = df_results.round(2)
        display(df_results.style.hide(axis='index').format(precision=2, thousands=','))
        
    else: 
        print(f"Training Data:\tR^2 = {r2_train:,.2f}\tRMSE = {rmse_train:,.2f}\tMAE = {mae_train:,.2f}")
        print(f"Test Data:\tR^2 = {r2_test:,.2f}\tRMSE = {rmse_test:,.2f}\tMAE = {mae_test:,.2f}")



## Model 1: Baseline LinReg

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg= LinearRegression()
lin_reg.fit(X_train_df,y_train)
evaluate_regression(lin_reg, X_train_df, y_train, 
                    X_test_df, y_test)

In [ ]:
de.plot_residuals(lin_reg, X_test_df,y_test)

### Extracting and Visualizing Coefficients

#### Extracting Coefficients

In [ ]:
## access the .coef_ 
# lin_reg.coef_

In [ ]:
## Intercept
# lin_reg.intercept_

In [ ]:
# ## Saving the coefficients
# coeffs = pd.Series(lin_reg.coef_, index= lin_reg.feature_names_in_)
# coeffs.loc['intercept'] = lin_reg.intercept_
# coeffs

#### def `get_coefficients`

In [ ]:
def get_coefficients(lin_reg):
    coeffs = pd.Series(lin_reg.coef_, index= lin_reg.feature_names_in_)
    coeffs.loc['intercept'] = lin_reg.intercept_
    return coeffs

In [ ]:
coeffs = get_coefficients(lin_reg)
coeffs

In [ ]:
## formatting numbers to not use , thousands sep, and 4 digits floats
pd.set_option('display.float_format',lambda x: f"{x:,.4f}")
pd.set_option('display.max_rows', None)

In [ ]:
coeffs.sort_values()

### Visualizing

#### **Q: How can we handle this many coefficients in our viz?**

- Some options include:
    - Only plot most the N most positive and N most negative.
    - Separate out OHE countries into a separate graph.
    
    
- Let's try the second option: separating the OHE countries.

In [ ]:
## Get a list of all of the ohe columsn
country_feats = [c for c in X_train_df.columns if c.startswith('Country')]
len(country_feats)

In [ ]:
## Plot country ceoffs
ax = coeffs[country_feats].sort_values().plot(kind='barh', figsize=(8,26))
ax.axvline(0, color='k')
ax.set(xlabel='Life Expectancy', title="Coefficients - Countries");

In [ ]:
## Plot everything but countries
ax = coeffs.drop(country_feats).sort_values().plot(kind='barh')#, figsize=(8,26))
ax.axvline(0)
ax.set(xlabel='Life Expectancy', title="Coefficients - Without Countries");

> ***Q: What do we notice about our non-country coefficients? Is there anything odd that would be difficult to a stakeholder?***

- What would we get if we didn't include an intercept?

## Model #2 - No Intercept

In [ ]:
lin_reg_noint= LinearRegression(fit_intercept=False)
lin_reg_noint.fit(X_train_df,y_train)
evaluate_regression(lin_reg_noint, X_train_df, y_train, 
                    X_test_df, y_test)

> Notice the model's performance is the same

In [ ]:
de.plot_residuals(lin_reg_noint, X_test_df,y_test)

In [ ]:
coeffs_noint = get_coefficients(lin_reg_noint)
coeffs_noint

> But coefficients have changed

In [ ]:
ax = coeffs_noint[country_feats].sort_values().plot(kind='barh', figsize=(8,26))
ax.axvline(0)
ax.set(xlabel='Life Expectancy', title="Coefficients - Countries");

In [ ]:
ax = coeffs_noint.drop(country_feats).sort_values().plot(kind='barh')#, figsize=(8,26))
ax.axvline(0)
ax.set(xlabel='Life Expectancy', title="Coefficients - Without Countries");

## Model #3 - No Countries

In [ ]:
# ### Train Test Split
## Make x and y variables
target = "Life expectancy"
drop_feats = ["Country"]

y = df[target].copy()
X = df.drop(columns=[target, *drop_feats]).copy()

## train-test-split with random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=SEED)

## Make numeric preprocessing pipeline
num_sel = make_column_selector(dtype_include='number')
num_pipe = make_pipeline(SimpleImputer(strategy='mean'))


## Make categorical preprocessing pipeline
cat_sel = make_column_selector(dtype_include='object')
cat_pipe = make_pipeline(SimpleImputer(strategy='constant',
                                       fill_value='MISSING'),
                         OneHotEncoder(handle_unknown='ignore', sparse=False))

In [ ]:
preprocessor.fit(X_train)
feature_names = preprocessor.get_feature_names_out()
X_train_df = pd.DataFrame(preprocessor.transform(X_train),
                          columns=feature_names,
                         index=X_train.index)
X_test_df = pd.DataFrame(preprocessor.transform(X_test),
                          columns=feature_names,
                         index=X_test.index)


In [ ]:
lin_reg= LinearRegression(fit_intercept=False)
lin_reg.fit(X_train_df,y_train)
evaluate_regression(lin_reg, X_train_df, y_train, 
                    X_test_df, y_test)

In [ ]:
de.plot_residuals(lin_reg, X_test_df,y_test)

In [ ]:
coeffs = get_coefficients(lin_reg)
coeffs

In [ ]:
coeffs.sort_values().plot(kind='barh')

## Model 3: Dropping Categories with OneHotEncoder

####  OneHotEncoder's Drop Argument

In [ ]:
## make pipelines for categorical vs numeric data
cat_pipe = make_pipeline(SimpleImputer(strategy='constant',
                                       fill_value='MISSING'),
                         OneHotEncoder(drop='if_binary', sparse=False,
                                      ))
cat_pipe

In [ ]:
num_pipe = make_pipeline(SimpleImputer(strategy='mean'))

## make the preprocessing column transformer
preprocessor = make_column_transformer((num_pipe, num_sel),
                                       (cat_pipe,cat_sel),
                                      verbose_feature_names_out=False)
preprocessor

preprocessor.fit(X_train)
feature_names = preprocessor.get_feature_names_out()

X_train_df = pd.DataFrame(preprocessor.transform(X_train),
                          columns=feature_names,
                         index=X_train.index)
X_test_df = pd.DataFrame(preprocessor.transform(X_test),
                          columns=feature_names,
                         index=X_test.index)
display(X_train_df.head())

In [ ]:
lin_reg_drop = LinearRegression(fit_intercept=False)
lin_reg_drop.fit(X_train_df,y_train)
evaluate_regression(lin_reg_drop, X_train_df, y_train, 
                    X_test_df, y_test)

de.plot_residuals(lin_reg_drop, X_test_df,y_test)

In [ ]:
coeffs_dropped = get_coefficients(lin_reg_drop)


ax = coeffs_dropped.sort_values().plot(kind='barh')#, figsize=(8,26))
ax.axvline(0)
ax.set(xlabel='Life Expectancy', title="Coefficients - Without Countries");

> ***Q: What are the most positive coefficients? What are the most negative?***

- Why does infant_deaths have a positive coefficient??

In [ ]:
plot_df = pd.concat([X_train_df,y_train],axis=1)

In [ ]:
sns.scatterplot(data=plot_df, x='infant deaths', y=target)

In [ ]:
regplot_kws = dict(line_kws={'color':'red'},
                  scatter_kws={'ec':'white','lw':0.5})
sns.regplot(data=plot_df, x='infant deaths', y=target,**regplot_kws)
           

In [ ]:
sns.regplot(data=plot_df, x='infant deaths', y=target,**regplot_kws,
           lowess=True)


> Feature is too complex for a simple linear regression. Let's try a random forest.

# BOOKMARK

## Feature Importance

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train_df,y_train)
evaluate_regression(rf_reg, X_train_df, y_train, 
                    X_test_df, y_test)

> Using the models .feature_names_in_

In [ ]:
## Saving the coefficients
importances = pd.Series(rf_reg.feature_importances_, index= rf_reg.feature_names_in_)
importances

In [ ]:
ax = importances.sort_values().tail(20).plot(kind='barh')#,figsize=(6,4))
ax.axvline(0, color='k')
ax.set(title='Feature Importance - Decision Tree Regressor',ylabel="Feature Name",
      xlabel='Importance');


## Using joblib to Save our Model, Data, and Objects

In [ ]:
X_train.head()

In [ ]:
evaluate_regression(lin_reg,X_train_df,y_train, X_test_df, y_test)

In [ ]:
evaluate_regression(rf_reg,X_train_df,y_train, X_test_df, y_test)

In [ ]:
import joblib, os

## creating a dictionary of all of the variables to save for later
export = {'X_train':X_train,
         'y_train':y_train,
         'X_test':X_test,
         'y_test':y_test,
          'preprocessor':preprocessor,
         'LinearRegression': lin_reg,
          'RandomForestRegressor':rf_reg
         }

In [ ]:
folder = "../Models/"
os.makedirs(folder, exist_ok=True)

In [ ]:
fname = folder+'wk1-lect01-codealong.joblib'
joblib.dump(export, fname)

In [ ]:
loaded = joblib.load(fname)
loaded.keys()

> We will continue working with this task and these models next class!

# *Teaser* Shap (For Regression)

In [ ]:
# Import and init shap
import shap
shap.initjs()

In [ ]:
# Take a sample of the training data
X_shap = shap.sample(X_train_df,nsamples = 500,random_state=SEED)
y_shap = y_train.loc[X_shap.index]

# Instantiate a Model Explainer with the model
explainer = shap.Explainer(rf_reg)

## Get shap values form the explainer
shap_values = explainer(X_shap,y_shap)

In [ ]:
shap.summary_plot(shap_values, features = X_shap)